# Install libraries 

In [ ]:
import pandas as pd
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
import warnings
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import time
import joblib

from sklearn.pipeline import Pipeline

# Read the Excel file into a Pandas DataFrame

In [ ]:

data = pd.read_csv("/kaggle/input/iot23-dataset/iot23_combined.csv")

In [ ]:
data.head()

In [ ]:
data.columns

# Show the unique values of the type of attacks

In [ ]:
data.label.unique()

# Count the frequencies of the type of attack

In [ ]:
data['label'].value_counts()

# Plot the frequencies of every type of attack

In [ ]:
sns.countplot(x='label', data=data)
plt.xticks(rotation=60)
plt.show()


## Clean the data

In [ ]:
data['label'].replace({'-   Benign   -': 'Benign', 'Benign ': 'Benign'}, inplace=True)


In [ ]:
data['label'].unique()

# Feature Engineering

In [ ]:
data['label'].replace({'C&C-HeartBeat': 'C&C',
                       'C&C-Torii': 'C&C',
                        'C&C-FileDownload': 'C&C',
                        'C&C-HeartBeat-FileDownload':'C&C',
                        'C&C-Mirai': 'C&C'}, inplace=True)


In [ ]:
data.label.unique()

In [ ]:
sns.countplot(x='label', data=data)
plt.xticks(rotation=60)
plt.show()


# Drop columns that are unessary

In [ ]:
del data['Unnamed: 0']

In [ ]:
del data['id.orig_h']

In [ ]:
del data['ts']

In [ ]:
data.head()

# Check for duplication

In [ ]:
# Step 1: Check for duplicated rows
duplicates = data.duplicated()

# Step 2: Count the occurrences of duplicates
duplicate_counts = duplicates.sum()
print("Number of duplicated rows:", duplicate_counts)


In [ ]:

# Step 3: Remove duplicated rows
data = data.drop_duplicates()

# Step 4: Verify that duplicates have been removed
duplicates_after_removal = data.duplicated().sum()
print("Number of duplicated rows after removal:", duplicates_after_removal)


In [ ]:
data

# Remove illogical data

In [ ]:
data.duration.unique()

In [ ]:

data = data[(data['duration'] >= 0)]

#  Print the resulting dataset
data.duration.unique()


# Save the cleaned data

In [ ]:
data.to_csv('/kaggle/working/cleaned_data.csv', index=False)

# Building The Classification Models

#### Classification models(Support Vector machine & Naive Bayes)

# Support Vector machine (SVM)

# Train the SVM model

In [ ]:
filepath = "/kaggle/input/iot23-dataset/cleaned_data.csv"
SVM = pd.read_csv(filepath, nrows=27000)

In [ ]:
SVM.shape

In [ ]:

# Define the features and target
features = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
target = 'label'

# Load the data
X = SVM[features]
Y = SVM[target]

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

# Create the SVM classifier model and fit it to the training data
SVM_classifier = SVC(C=1.0, cache_size=1500, verbose=True)
SVM_classifier.fit(X_train, Y_train)

# Evaluate the model on the testing data
score = SVM_classifier.score(X_test, Y_test)
print('Model Accuracy: ', score)
print()

# Predict the labels of the test set
y_pred = SVM_classifier.predict(X_test)

# Suppress the UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.metrics')

# Calculate the classification report of the model
report = classification_report(Y_test, y_pred, zero_division=0)
print('Classification Report:')
print(report)
print()

# Calculate the time taken by the model to run
start_time = time.time()
print()
end_time = time.time()
print('Time taken: ', end_time - start_time, 'seconds')


# Train the Naive Bayse model

### Read the data

In [ ]:
filepath = "/kaggle/input/iot23-dataset/cleaned_data.csv"

NP = pd.read_csv(filepath, nrows=27000)

In [ ]:

X = NP[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = NP['label']

# Normalize the features
scaler = MinMaxScaler()
normalized_X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(normalized_X, Y, random_state=100, test_size=0.2)

# Suppress the UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.metrics')

clf = GaussianNB()

start = time.time()
print('program start...')
print()

clf.fit(X_train, Y_train)
print()
print(clf.score(X_test, Y_test))
print()

y_pred = clf.predict(X_test)
print(y_pred)
print()

end = time.time()

print(end - start, 'seconds')

print("Classification Report:")
print(classification_report(Y_test, y_pred))




## Classification models( Decision Tree & Neural Networks)

### Read the dataframe


In [ ]:
filepath = "/kaggle/input/iot23-dataset/cleaned_data.csv"
DS = pd.read_csv(filepath, nrows=27000)

# Train Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time
import warnings

# Select features and target variable
features = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
target = 'label'

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(DS[features], DS[target], random_state=5, test_size=0.2)

# Create Decision Tree classifier
classifier = DecisionTreeClassifier()

start = time.time()
print('program start...')
print()

# Fit the model
classifier.fit(X_train, Y_train)
print()

# Evaluate the model
score = classifier.score(X_test, Y_test)
print(score)
print()
y_pred = classifier.predict(X_test)
print(y_pred)
print()

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

# Suppress the UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.metrics')

print("Classification Report:")
print(classification_report(Y_test, y_pred))


# Train CNN  Model

In [ ]:
filepath = "/kaggle/input/iot23-dataset/cleaned_data.csv"
CNN = pd.read_csv(filepath)


# Set features

In [ ]:
X = CNN[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']].values

# Encode the target variable

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
CNN['label'] = le.fit_transform(CNN['label'])


In [ ]:
CNN['label']

In [ ]:
CNN['label'].unique()

In [ ]:
#define the features and target
features = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
target = 'label'

#load the data
X = CNN[features]
Y = CNN[target]

In [ ]:
Y = pd.get_dummies(CNN['label']).values

In [ ]:
X.shape

# Scale the features

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X)

In [ ]:
normalized_x = scaler.transform(X)

In [ ]:
normalized_x

In [ ]:
normalized_x.shape

In [ ]:
scaler.fit(Y)

In [ ]:
normalized_y = scaler.transform(Y)

In [ ]:
normalized_y

# Conficure the archetucture of the CNN Model

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [ ]:
model = Sequential()

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# Define the features and target
features = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts',
            'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ',
            'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0',
            'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
target = 'label'


# Encode the target variable
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])

# Split the data into train and test sets
X = data[features]
Y = data[target]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Scale the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the target variable to categorical
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

# Configure the architecture of the CNN model
model = Sequential()
model.add(Dense(2000, activation='relu', input_dim=len(features)))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
start = time.time()
print('Program start...')
print()

history = model.fit(X_train, Y_train, epochs=10, batch_size=256, validation_data=(X_test, Y_test), verbose=1)
print()

end = time.time()
print('Program end...')
print()
print('Time cost: ')
print(end - start, 'seconds')

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
print('Test Accuracy:', test_accuracy)


# Save the model with the highest accuracy 

Since the desition tree model is the model with the highest classification accuracy, we will save it as .pkl

# Save it into a pkl

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# Select features and target variable
features = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
target = 'label'

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(DS[features], DS[target], random_state=5, test_size=0.2)

# Create pipeline with Decision Tree classifier
pipeline = Pipeline([
    ('classifier', DecisionTreeClassifier())
])

# Fit the model using the pipeline
pipeline.fit(X_train, Y_train)

# Evaluate the model
accuracy = pipeline.score(X_test, Y_test)
print("Accuracy:", accuracy)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(Y_test, y_pred))

# Save the pipeline as a .pkl file
filename = 'decision_tree_pipeline.pkl'
joblib.dump(pipeline, filename)
print("Pipeline saved as", filename)
